Goal: to identify which customers are likely to churn to implement some strategies to prevent it from happening

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report   
import seaborn as sns
import matplotlib.pyplot as plt
# set display rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# load data
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
df.head(1).T

,0
customerID,7590-VHVEG
gender,Female
SeniorCitizen,0
Partner,Yes
Dependents,No
tenure,1
PhoneService,No
MultipleLines,No phone service
InternetService,DSL
OnlineSecurity,No


In [6]:
# lower case columns
df.columns = df.columns.str.lower().str.replace(' ', '_')
# lower case values in object columns
for col in df.select_dtypes('object').columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [7]:
df.head(1)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no


In [8]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [9]:
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')

In [10]:
df[df.totalcharges.isnull()][['customerid', 'totalcharges']]

,customerid,totalcharges
488,4472-lvygi,NaN
753,3115-czmzd,NaN
936,5709-lvoeq,NaN
1082,4367-nuyao,NaN
1340,1371-dwpaz,NaN
3331,7644-omvmy,NaN
3826,3213-vvolg,NaN
4380,2520-sgtta,NaN
5218,2923-arzlg,NaN
6670,4075-wkniu,NaN


In [11]:
# fill na with 0, which could be not best approach but is ok for now as it will ignore the feature when modeling
df['totalcharges'] = df['totalcharges'].fillna(0)

In [12]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                object
dtype: object

In [13]:
df.head(1).T

,0
customerid,7590-vhveg
gender,female
seniorcitizen,0
partner,yes
dependents,no
tenure,1
phoneservice,no
multiplelines,no_phone_service
internetservice,dsl
onlinesecurity,no


In [14]:
# the churn column is our target and is object now, should convert to int
df['churn'] = (df['churn'] == 'yes').astype(int)

# Setup Validation Dataset

In [15]:
from sklearn.model_selection import train_test_split
# split the data
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
# reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

In [16]:
len(df_train), len(df_val), len(df_test), len(df_full_train)

(4225, 1409, 1409, 5634)

In [17]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values
y_full_train = df_full_train.churn.values

# EDA

- Check missing values
- Look at the target var
- Look at numerical and categorical var

In [18]:
df.isnull().sum().sum()

np.int64(0)

In [19]:
df.churn.value_counts(normalize=True)

churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

In [20]:
# mean is also churn rate as churn is binary values
global_churn_rate = df.churn.mean()
global_churn_rate.round(2)

np.float64(0.27)

In [21]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [22]:
categorical = [col for col in df.columns if col not in numerical + ['customerid']]

In [23]:
df[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
churn               2
dtype: int64

In [24]:
# look at the values of categorical variables
for col in categorical:
    print(f'\n{col}')
    print(df[col].value_counts())


gender
gender
male      3555
female    3488
Name: count, dtype: int64

seniorcitizen
seniorcitizen
0    5901
1    1142
Name: count, dtype: int64

partner
partner
no     3641
yes    3402
Name: count, dtype: int64

dependents
dependents
no     4933
yes    2110
Name: count, dtype: int64

phoneservice
phoneservice
yes    6361
no      682
Name: count, dtype: int64

multiplelines
multiplelines
no                  3390
yes                 2971
no_phone_service     682
Name: count, dtype: int64

internetservice
internetservice
fiber_optic    3096
dsl            2421
no             1526
Name: count, dtype: int64

onlinesecurity
onlinesecurity
no                     3498
yes                    2019
no_internet_service    1526
Name: count, dtype: int64

onlinebackup
onlinebackup
no                     3088
yes                    2429
no_internet_service    1526
Name: count, dtype: int64

deviceprotection
deviceprotection
no                     3095
yes                    2422
no_internet_service

In [25]:
# look at churn rate by categorical variables
for col in categorical:
    print(f'\n{col}')
    print(df.groupby(col).churn.mean().sort_values(ascending=False).round(2))


gender
gender
female    0.27
male      0.26
Name: churn, dtype: float64

seniorcitizen
seniorcitizen
1    0.42
0    0.24
Name: churn, dtype: float64

partner
partner
no     0.33
yes    0.20
Name: churn, dtype: float64

dependents
dependents
no     0.31
yes    0.15
Name: churn, dtype: float64

phoneservice
phoneservice
yes    0.27
no     0.25
Name: churn, dtype: float64

multiplelines
multiplelines
yes                 0.29
no                  0.25
no_phone_service    0.25
Name: churn, dtype: float64

internetservice
internetservice
fiber_optic    0.42
dsl            0.19
no             0.07
Name: churn, dtype: float64

onlinesecurity
onlinesecurity
no                     0.42
yes                    0.15
no_internet_service    0.07
Name: churn, dtype: float64

onlinebackup
onlinebackup
no                     0.40
yes                    0.22
no_internet_service    0.07
Name: churn, dtype: float64

deviceprotection
deviceprotection
no                     0.39
yes                    0.23
n

In [26]:
print(global_churn_rate)

0.2653698707936959


In [31]:
from IPython.display import display
for col in categorical:
    df_group = df_full_train.groupby(col).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn_rate
    df_group['risk'] = df_group['mean'] / global_churn_rate
    display(df_group)

,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.011454,1.043163
male,0.263214,2838,-0.002156,0.991874


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.023100,0.912953
1,0.413377,912,0.148007,1.557740


,mean,count,diff,risk
partner,,,,
no,0.329809,2932,0.064439,1.242828
yes,0.205033,2702,-0.060337,0.772632


,mean,count,diff,risk
dependents,,,,
no,0.313760,3968,0.048390,1.182350
yes,0.165666,1666,-0.099704,0.624284


,mean,count,diff,risk
phoneservice,,,,
no,0.241316,547,-0.024054,0.909358
yes,0.273049,5087,0.007679,1.028937


...

,mean,count,diff,risk
streamingmovies,,,,
no,0.338906,2213,0.073537,1.277110
no_internet_service,0.077805,1221,-0.187565,0.293195
yes,0.307273,2200,0.041903,1.157904


,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.166331,1.626790
one_year,0.120573,1186,-0.144797,0.454360
two_year,0.028274,1344,-0.237096,0.106545


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.093299,0.648419
yes,0.338151,3321,0.072781,1.274264


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.097199,0.633722
credit_card_(automatic),0.164339,1217,-0.101031,0.619281
electronic_check,0.455890,1893,0.190520,1.717942
mailed_check,0.193870,1305,-0.071500,0.730564


,mean,count,diff,risk
churn,,,,
0,0.0,4113,-0.26537,0.000000
1,1.0,1521,0.73463,3.768325


# Feature Importance - Mutual Information

In [32]:
from sklearn.metrics import mutual_info_score
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)   

In [34]:
df_full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending=False)

churn               0.583227
contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64